## **Conformal Prediction using Energy Hospital Load**

One example of the NP data is the electricity consumption of a hospital in SF. It has hourly reservation for the entire year of 2015.

### Extract Data and Split Data into Train, Val, Cal, and Test


**Extract Data From GitHub**

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from neuralprophet import NeuralProphet, set_log_level, set_random_seed

In [2]:
data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
file = 'energy/SF_hospital_load.csv'
# file = 'air_passengers.csv'

In [3]:
data_df = pd.read_csv(data_location + file)

In [4]:
# data_df.head(5)

In [5]:
# data_df.tail(5)

**Split data into train and test in that order**

Do we need to instantiate a NP model `m` in order to split the df into train and test? If so, does the NP params make any diff to the outcome of this split? I'm presuming no atm.

In [6]:
train_df, test_df = NeuralProphet().split_df(data_df, freq='H', valid_p = 1.0/16)
train_df.shape, test_df.shape

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.989% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


((8213, 2), (547, 2))

In [7]:
print(f"Train set time range:        {train_df['ds'].min()} - {train_df['ds'].max()}")
print(f"Test set time range:         {test_df['ds'].min()} - {test_df['ds'].max()}")

Train set time range:        2015-01-01 01:00:00 - 2015-12-09 05:00:00
Test set time range:         2015-12-09 06:00:00 - 2016-01-01 00:00:00


### Create Folds using CV Splits

In [8]:
B = 25                # Number of bootstraps and number of k-folds
val_fold_pct = 0.05   # Validation fold size % of entire input
val_cov_pct = 0.5     # Overall validation set coverage %
val_fold_size = 0.1  # Fixed validation fold size % of entire fold (hence train fold size is 1 - val_fold_size) 
print(f"Fold overlap proportion: {round(B*val_fold_pct - val_cov_pct, 2)}")

Fold overlap proportion: 0.75


In [9]:
folds = NeuralProphet().crossvalidation_split_df(
    train_df,
    freq='H',
    k=B,
    fold_pct=val_fold_pct,
    fold_overlap_pct=(B*val_fold_pct - val_cov_pct)
)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.988% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


In [10]:
for i, (train_fold_df, cal_fold_df) in enumerate(folds):
    print(f"Fold {i+1}:")
    if val_fold_size:
        train_fold_len = int(len(cal_fold_df) * ((1-val_fold_size)/val_fold_size))
        train_fold_df = train_fold_df[-train_fold_len: ]
    print(f"  - Train start: {train_fold_df.ds.min()}, Train end: {train_fold_df.ds.max()}")
    print(f"  - Cal start:   {cal_fold_df.ds.min()}, Cal end:   {cal_fold_df.ds.max()}")
    print(f"  - Train shape:   {train_fold_df.shape}, Cal shape:   {cal_fold_df.shape}")

Fold 1:
  - Train start: 2015-03-10 10:00:00, Train end: 2015-08-11 03:00:00
  - Cal start:   2015-08-11 04:00:00, Cal end:   2015-08-28 05:00:00
  - Train shape:   (3690, 2), Cal shape:   (410, 2)
Fold 2:
  - Train start: 2015-03-14 17:00:00, Train end: 2015-08-15 10:00:00
  - Cal start:   2015-08-15 11:00:00, Cal end:   2015-09-01 12:00:00
  - Train shape:   (3690, 2), Cal shape:   (410, 2)
Fold 3:
  - Train start: 2015-03-19 00:00:00, Train end: 2015-08-19 17:00:00
  - Cal start:   2015-08-19 18:00:00, Cal end:   2015-09-05 19:00:00
  - Train shape:   (3690, 2), Cal shape:   (410, 2)
Fold 4:
  - Train start: 2015-03-23 07:00:00, Train end: 2015-08-24 00:00:00
  - Cal start:   2015-08-24 01:00:00, Cal end:   2015-09-10 02:00:00
  - Train shape:   (3690, 2), Cal shape:   (410, 2)
Fold 5:
  - Train start: 2015-03-27 14:00:00, Train end: 2015-08-28 07:00:00
  - Cal start:   2015-08-28 08:00:00, Cal end:   2015-09-14 09:00:00
  - Train shape:   (3690, 2), Cal shape:   (410, 2)
Fold 6:
  

### Aggregation of Calibration data without Prediction

In [11]:
cal_agg_df = pd.DataFrame( )

In [12]:
for _, cal_fold_df in folds:
    cal_agg_df = pd.concat([cal_agg_df, cal_fold_df], ignore_index=True)

In [13]:
cal_agg_df

,ds,y
0,2015-08-11 04:00:00,907.150481
1,2015-08-11 05:00:00,915.825992
2,2015-08-11 06:00:00,1130.377543
3,2015-08-11 07:00:00,1229.058650
4,2015-08-11 08:00:00,1260.755035
...,...,...
10245,2015-12-09 01:00:00,795.054930
10246,2015-12-09 02:00:00,796.664865
10247,2015-12-09 03:00:00,795.577339
10248,2015-12-09 04:00:00,793.961047


In [14]:
# cal_agg_df.groupby('ds').count().reset_index().sort_values(by='y', ascending=False)

In [15]:
cal_agg_df = cal_agg_df.groupby('ds').mean().reset_index()

In [16]:
cal_agg_df

,ds,y
0,2015-08-11 04:00:00,907.150481
1,2015-08-11 05:00:00,915.825992
2,2015-08-11 06:00:00,1130.377543
3,2015-08-11 07:00:00,1229.058650
4,2015-08-11 08:00:00,1260.755035
...,...,...
2877,2015-12-09 01:00:00,795.054930
2878,2015-12-09 02:00:00,796.664865
2879,2015-12-09 03:00:00,795.577339
2880,2015-12-09 04:00:00,793.961047
